In [59]:
import os, json, uuid, base64, re
from typing import List, Dict, Tuple, TypedDict, Literal, Optional
from dataclasses import dataclass
from pathlib import Path

import numpy as np
import pandas as pd
import faiss

from pypdf import PdfReader

import pymupdf

# 환경변수 생성하기
from dotenv import load_dotenv
load_dotenv()

# LangGraph
from langgraph.graph import StateGraph, START, END

# 
from openai import OpenAI
client = OpenAI()  # 환경변수 OPENAI_API_KEY 필요

EMBED_MODEL = "text-embedding-3-small"   # 1536차원, 비용 대비 성능 우수
GEN_MODEL   = "gpt-4o-mini"              # 멀티모달(텍스트+이미지) 추론 모델
TOP_K = 10

In [60]:
def split_pdf(filepath, batch_size=10):
    """
    입력 PDF를 여러 개의 작은 PDF 파일로 분할
    """
    # PDF 파일 열기
    input_pdf = pymupdf.open(filepath)
    num_pages = len(input_pdf)
    print(f"총 페이지 수: {num_pages}")

    ret = []
    # PDF 분할
    for start_page in range(0, num_pages, batch_size):
        end_page = min(start_page + batch_size, num_pages) - 1

        # 분할된 PDF 저장
        input_file_basename = os.path.splitext(filepath)[0]
        output_file = f"{input_file_basename}_{start_page:04d}_{end_page:04d}.pdf"
        print(f"분할 PDF 생성: {output_file}")

        with pymupdf.open() as output_pdf:
            output_pdf.insert_pdf(input_pdf, from_page=start_page, to_page=end_page)
            output_pdf.save(output_file)
            ret.append(output_file)

    # 입력 PDF 파일 닫기
    input_pdf.close()
    return ret

In [61]:
import os
import json
import requests
import pymupdf

class LayoutAnalyzer:
    def __init__(self, api_key):
        self.api_key = api_key

    def _upstage_layout_analysis(self, input_file):
        """Upstage API 호출 후 결과 JSON 저장"""
        response = requests.post(
            "https://api.upstage.ai/v1/document-ai/layout-analysis",
            headers={"Authorization": f"Bearer {self.api_key}"},
            data={"ocr": False},
            files={"document": open(input_file, "rb")},
        )

        if response.status_code == 200:
            output_file = os.path.splitext(input_file)[0] + ".json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(response.json(), f, ensure_ascii=False, indent=2)
            return output_file
        else:
            raise ValueError(f"예상치 못한 상태 코드: {response.status_code}")

    def execute(self, input_file):
        return self._upstage_layout_analysis(input_file)


def split_pdf(filepath, batch_size=10, output_dir=None):
    """PDF를 batch_size 단위로 분할 저장 (자동 폴더 생성 포함)"""
    input_pdf = pymupdf.open(filepath)
    num_pages = len(input_pdf)
    print(f"총 페이지 수: {num_pages} ({os.path.basename(filepath)})")

    base_name = os.path.splitext(os.path.basename(filepath))[0]

    # ✅ PDF별 개별 하위 폴더 자동 생성
    if output_dir is None:
        parent_dir = os.path.dirname(filepath)
        output_dir = os.path.join(parent_dir, base_name)
    os.makedirs(output_dir, exist_ok=True)

    ret = []
    for start_page in range(0, num_pages, batch_size):
        end_page = min(start_page + batch_size, num_pages) - 1
        output_file = os.path.join(
            output_dir, f"{base_name}_{start_page:04d}_{end_page:04d}.pdf"
        )
        print(f"  ➤ 분할 PDF 생성: {output_file}")

        with pymupdf.open() as output_pdf:
            output_pdf.insert_pdf(input_pdf, from_page=start_page, to_page=end_page)
            output_pdf.save(output_file)
            ret.append(output_file)

    input_pdf.close()
    return ret


def analyze_pdf(filepath, api_key, batch_size=10, output_dir=None):
    """PDF를 분할 → 각 조각별 레이아웃 분석 실행 → JSON 파일 리스트 반환"""
    analyzer = LayoutAnalyzer(api_key)
    split_files = split_pdf(filepath, batch_size=batch_size, output_dir=output_dir)

    analyzed_files = []
    for pdf_file in split_files:
        print(f"  ➤ 분석 중: {pdf_file}")
        try:
            result_json = analyzer.execute(pdf_file)
            analyzed_files.append(result_json)
        except Exception as e:
            print(f"  ⚠️ 오류 발생 ({pdf_file}): {e}")

    return analyzed_files


def analyze_folder(folder_path, api_key, batch_size=10, output_dir=None):
    """폴더 내 모든 PDF 파일을 순차적으로 분석 (하위폴더 자동 생성)"""
    pdf_files = [
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if f.lower().endswith(".pdf")
    ]

    print(f"총 {len(pdf_files)}개의 PDF 파일을 발견했습니다.")
    all_results = []

    for pdf_file in pdf_files:
        print(f"\n==== {os.path.basename(pdf_file)} 분석 시작 ====")
        results = analyze_pdf(pdf_file, api_key, batch_size=batch_size, output_dir=output_dir)
        all_results.extend(results)

    return all_results


# 실행 예시
if __name__ == "__main__":
    api_key = "up_2JFq4Gar1LUIIOO4cFlMYrTFwoUzs" # api 코드 변경
    pdf_folder = "/home/user/langchain-study/2024_수요보고서" # 해당 내용만 수정하여 처리

    analyzed_files = analyze_folder(pdf_folder, api_key, batch_size=10)
    print("\n전체 분석 완료. 생성된 JSON 파일 목록:")
    for f in analyzed_files:
        print(" -", f)

총 12개의 PDF 파일을 발견했습니다.

==== 2024년7월전력수요예측전망.pdf 분석 시작 ====
총 페이지 수: 8 (2024년7월전력수요예측전망.pdf)
  ➤ 분할 PDF 생성: /home/user/langchain-study/2024_수요보고서/2024년7월전력수요예측전망/2024년7월전력수요예측전망_0000_0007.pdf
  ➤ 분석 중: /home/user/langchain-study/2024_수요보고서/2024년7월전력수요예측전망/2024년7월전력수요예측전망_0000_0007.pdf

==== 2024년12월전력수요예측전망.pdf 분석 시작 ====
총 페이지 수: 7 (2024년12월전력수요예측전망.pdf)
  ➤ 분할 PDF 생성: /home/user/langchain-study/2024_수요보고서/2024년12월전력수요예측전망/2024년12월전력수요예측전망_0000_0006.pdf
  ➤ 분석 중: /home/user/langchain-study/2024_수요보고서/2024년12월전력수요예측전망/2024년12월전력수요예측전망_0000_0006.pdf

==== 2024년8월전력수요예측전망.pdf 분석 시작 ====
총 페이지 수: 8 (2024년8월전력수요예측전망.pdf)
  ➤ 분할 PDF 생성: /home/user/langchain-study/2024_수요보고서/2024년8월전력수요예측전망/2024년8월전력수요예측전망_0000_0007.pdf
  ➤ 

In [63]:
# -*- coding: utf-8 -*-
import os
import re
import glob
import json
import urllib.parse
import pymupdf
from PIL import Image, ImageDraw

# =========================
# 추출기 (네 로직 유지 + 보강)
# =========================
class SimpleWorkingExtractor:
    """단순하지만 잘 작동하는 추출기 (텍스트 + 표→Markdown + figure/chart 이미지 저장 + 디버그 박스)"""
    def __init__(self, pdf_file):
        self.pdf_file = pdf_file

    def extract_simple(self, json_file, output_dir="simple_output"):
        os.makedirs(output_dir, exist_ok=True)

        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        meta_pages = {p.get('page'): (p.get('width'), p.get('height'))
                      for p in data.get('metadata', {}).get('pages', [])}

        # 파일명 끝의 _0000_0009 패턴에서 시작 페이지 추정 (split_pdf 기준 0-based → +1)
        filename = os.path.basename(json_file)
        start_page = 1
        m = re.search(r'_(\d{4})_(\d{4})\.json$', filename)
        if m:
            try:
                start_page = int(m.group(1)) + 1
            except Exception:
                start_page = 1

        elements = data.get('elements', [])
        visual_elements = [e for e in elements if e.get('category') in
                           ['table','figure','chart','text','header','footer',
                            'paragraph','heading1','heading2','list','index']]

        if not visual_elements:
            print(f"⚠️ 시각적 요소 없음: {json_file}")
            return []

        results = []
        with pymupdf.open(self.pdf_file) as doc:
            for idx, element in enumerate(visual_elements, 1):
                category = element.get('category', 'unknown')
                coords = element.get('bounding_box', [])

                # 텍스트
                if category in ['text','header','footer','paragraph','heading1','heading2','list','index']:
                    text_content = (element.get("text") or "").strip()
                    if text_content:
                        results.append({"type":"text","category":category,"content":text_content})
                    continue

                # 표 → Markdown
                if category == "table":
                    table_text = (element.get("text") or "").strip()
                    if table_text:
                        rows = [r.strip() for r in table_text.split("\n") if r.strip()]
                        md_lines = []
                        for i, row in enumerate(rows):
                            cells = [c for c in row.split() if c]
                            if not cells: continue
                            md_lines.append("| " + " | ".join(cells) + " |")
                            if i == 0:
                                md_lines.append("| " + " | ".join(["---"]*len(cells)) + " |")
                        if md_lines:
                            results.append({"type":"table","category":"table","markdown":"\n".join(md_lines)})

                if not coords:
                    continue

                json_page = element.get('page', 1)
                actual_page = start_page + json_page - 2  # 둘 다 1-based → 0-based 보정
                if actual_page < 0 or actual_page >= len(doc):
                    continue

                page = doc[actual_page]
                pix = page.get_pixmap(dpi=300)
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                img_w, img_h = img.size

                meta_w, meta_h = meta_pages.get(json_page, (page.rect.width, page.rect.height))
                meta_w = meta_w or page.rect.width
                meta_h = meta_h or page.rect.height
                scale_x = float(img_w)/float(meta_w) if meta_w else 1.0
                scale_y = float(img_h)/float(meta_h) if meta_h else 1.0

                xs = [c.get('x',0) for c in coords]
                ys = [c.get('y',0) for c in coords]
                if not xs or not ys:
                    continue
                x1,x2 = min(xs), max(xs)
                y1,y2 = min(ys), max(ys)

                left   = int(x1*scale_x); top    = int(y1*scale_y)
                right  = int(x2*scale_x); bottom = int(y2*scale_y)

                padding = 40 if category=='table' else 30 if category in ['figure','chart'] else 25
                crop_left   = max(0, left  - padding)
                crop_top    = max(0, top   - padding)
                crop_right  = min(img_w, right + padding)
                crop_bottom = min(img_h, bottom+ padding)

                # 모서리 보정
                if left < 100:               crop_left  = max(0, left  - int(padding*1.5))
                if top < 100:                crop_top   = max(0, top   - int(padding*1.5))
                if right > img_w - 100:      crop_right = min(img_w, right + int(padding*1.5))
                if bottom > img_h - 100:     crop_bottom= min(img_h, bottom+ int(padding*1.5))

                crop_box = (int(crop_left), int(crop_top), int(crop_right), int(crop_bottom))
                if crop_box[2] <= crop_box[0] or crop_box[3] <= crop_box[1]:
                    continue

                cropped_img = img.crop(crop_box)
                base_name = os.path.splitext(os.path.basename(json_file))[0]
                out_img = os.path.join(output_dir, f"{base_name}_{category}_{idx}.png")
                cropped_img.save(out_img)

                # 디버그 이미지 (선택)
                try:
                    dbg = img.copy()
                    draw = ImageDraw.Draw(dbg)
                    colors = {'table':'blue','figure':'green','chart':'orange'}
                    draw.rectangle([left,top,right,bottom], outline=colors.get(category,'red'), width=2)
                    draw.rectangle(crop_box, outline="red", width=3)
                    draw.text((crop_box[0]+5, max(0, crop_box[1]-25)), f"{category.upper()} {idx}", fill="red")
                    dbg.save(os.path.join(output_dir, f"{base_name}_{category}_{idx}_debug.png"))
                except Exception:
                    pass

                results.append({"type":"image","category":category,"path":out_img})

        print(f"✅ 추출 완료: {len(results)}개 요소 ({os.path.basename(json_file)})")
        return results

# ====================================
# JSON ↔ PDF 매칭 유틸 (재귀 폴더 대응)
# ====================================
def strip_page_range_from_stem(stem: str) -> str:
    m = re.search(r'_(\d{4})_(\d{4})$', stem)
    return stem[:m.start()] if m else stem

def resolve_pdf_for_json(json_path: str, pdf_root: str) -> str|None:
    """JSON 이름에서 분할 접미사 제거한 base로 원본 PDF 탐색 (exact → startswith → 느슨한 startswith)"""
    stem = os.path.splitext(os.path.basename(json_path))[0]
    base_stem = strip_page_range_from_stem(stem)

    exact = os.path.join(pdf_root, base_stem + ".pdf")
    if os.path.isfile(exact):
        return exact

    # startswith (버전/괄호/공백 등 허용)
    candidates = glob.glob(os.path.join(pdf_root, base_stem + "*.pdf"))
    if candidates:
        return max(candidates, key=os.path.getsize)

    # 느슨한 startswith
    pdfs = glob.glob(os.path.join(pdf_root, "*.pdf"))
    for p in pdfs:
        if os.path.splitext(os.path.basename(p))[0].startswith(base_stem):
            return p
    return None

# =========================
# 메인: 재귀 탐색 + 일괄 처리
# =========================
if __name__ == "__main__":
    print("📄 JSON 재귀 수집 → 원본 PDF 매칭 → 추출 → 요약(MD/HTML) 생성")

    # 너의 구조에 맞게 설정
    pdf_root    = "/home/user/langchain-study/2024_수요보고서"      # 원본 PDF 폴더
    json_root   = "/home/user/langchain-study/2024_수요보고서"      # JSON이 들어있는 폴더(하위 디렉토리 포함)
    output_root = "/home/user/langchain-study/2024_visuals_output"
    os.makedirs(output_root, exist_ok=True)

    # ✅ 재귀적으로 모든 JSON 수집
    json_files = sorted(glob.glob(os.path.join(json_root, "**", "*.json"), recursive=True))
    if not json_files:
        print(f"⚠️ JSON 파일을 찾지 못했습니다: {json_root}")
        raise SystemExit(1)

    all_results = []

    for json_path in json_files:
        json_name = os.path.basename(json_path)
        base_stem = strip_page_range_from_stem(os.path.splitext(json_name)[0])

        # 원본 PDF 찾기
        pdf_path = resolve_pdf_for_json(json_path, pdf_root)
        if not pdf_path:
            print(f"⚠️ 원본 PDF 매칭 실패: {json_name} (base: {base_stem})")
            continue

        pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
        print(f"\n==== 처리 시작 ====\nJSON: {json_name}\nPDF : {pdf_name}")

        # PDF별 출력 폴더
        pdf_output_dir = os.path.join(output_root, pdf_name)
        os.makedirs(pdf_output_dir, exist_ok=True)

        extractor = SimpleWorkingExtractor(pdf_path)
        try:
            results = extractor.extract_simple(json_path, pdf_output_dir)
            # 어느 PDF에서 나온 결과인지 태깅
            all_results.extend([dict(item, **{"__pdf__": pdf_name}) for item in results])
        except Exception as e:
            print(f"  ⚠️ 추출 중 오류: {e}")
            continue

    # ===============================
    # 전체 요약 MD/HTML 생성
    # ===============================
    md_path = os.path.join(output_root, "2024_summary.md")
    html_path = os.path.join(output_root, "2024_summary.html")

    with open(md_path, "w", encoding="utf-8") as f_md, open(html_path, "w", encoding="utf-8") as f_html:
        f_md.write("# 📊 2024년 전력수요 보고서 전체 추출 결과\n\n")
        f_html.write("<html><head><meta charset='utf-8'><title>2024 전력수요 요약</title></head><body>\n")
        f_html.write("<h1>📊 2024년 전력수요 보고서 전체 추출 결과</h1>\n")

        for item in all_results:
            src_pdf = item.get("__pdf__", "")
            if item["type"] == "image" and item["category"] in ["figure","chart"]:
                rel_path = os.path.relpath(item["path"], output_root)
                rel_path_encoded = urllib.parse.quote(rel_path)
                f_md.write(f"### 📄 {src_pdf}\n\n")
                f_md.write(f"![{os.path.basename(item['path'])}]({rel_path_encoded})\n\n")
                f_html.write(f"<h3>📄 {src_pdf}</h3>\n")
                f_html.write(f"<div style='margin:14px 0;'><img src='{rel_path_encoded}' style='max-width:100%;border:1px solid #ccc;'></div>\n")
            elif item["type"] == "text":
                f_md.write(f"**[{src_pdf}]** {item['content']}\n\n")
                safe_text = item["content"].replace("\n","<br>")
                f_html.write(f"<p><b>[{src_pdf}]</b> {safe_text}</p>\n")
            elif item["type"] == "table":
                f_md.write(f"**[{src_pdf}]**\n{item['markdown']}\n\n")
                f_html.write(f"<div style='margin:16px 0;'><pre>{item['markdown']}</pre></div>\n")

        f_html.write("</body></html>")

    print("\n✅ 완료! 재귀 탐색으로 모든 JSON을 처리하고, 매칭된 PDF 기준으로 추출/요약을 생성했습니다.")
    print(f"📄 Markdown 요약: {md_path}")
    print(f"🌐 HTML 요약   : {html_path}")


📄 JSON 재귀 수집 → 원본 PDF 매칭 → 추출 → 요약(MD/HTML) 생성

==== 처리 시작 ====
JSON: 2024년10월전력수요예측전망_0000_0007.json
PDF : 2024년10월전력수요예측전망
✅ 추출 완료: 57개 요소 (2024년10월전력수요예측전망_0000_0007.json)

==== 처리 시작 ====
JSON: 2024년11월전력수요예측전망_0000_0007.json
PDF : 2024년11월전력수요예측전망
✅ 추출 완료: 60개 요소 (2024년11월전력수요예측전망_0000_0007.json)

==== 처리 시작 ====
JSON: 2024년12월전력수요예측전망_0000_0006.json
PDF : 2024년12월전력수요예측전망
✅ 추출 완료: 56개 요소 (2024년12월전력수요예측전망_0000_0006.json)

==== 처리 시작 ====
JSON: 2024년1월전력수요예측전망_v1_0000_0007.json
PDF : 2024년1월전력수요예측전망_v1
✅ 추출 완료: 88개 요소 (2024년1월전력수요예측전망_v1_0000_0007.json)

==== 처리 시작 ====
JSON: 2024년2월전력수요예측전망_0000_0007.json
PDF : 2024년2월전력수요예측전망
✅ 추출 완료: 86개 요소 (2024년2월전력수요예측전망_0000_0007.json)

==== 처리 시작 ====
JSON: 2024년3월전력수요예측전망

## 랭그래프를 이용한 코드 

In [64]:
from typing import TypedDict

class GraphState(TypedDict):
    filepath: str
    filetype: str
    page_numbers: list[int]
    batch_size: int
    split_filepaths: list[str]
    analyzed_files: list[str]
    page_elements: dict[int, dict[str, list[dict]]]
    page_metadata: dict[int, dict]
    page_summary: dict[int, str]
    images: dict[int, list[dict]]           # figure, chart만
    images_summary: dict[int, list[dict]]   # figure, chart 해석만
    tables: dict[int, list[dict]]           # 통합: 마크다운 + 이미지 + 해석
    texts: dict[int, list[dict]]
    texts_summary: dict[int, dict]


# 변수 지정
# 실제 state 초기화
state: GraphState = {
    "filepath": "", # 단일 파일 경로를 가져가는 경우 처리
    "filetype": "pdf", # 파일형태를 채우는 함수
    "page_numbers": [], # 페이지 번호를 채우는 함수
    "batch_size": 4,
    "split_filepaths": [], # 
    "analyzed_files": [], #  extract_page_elements에서 부여
    "page_elements": {},  # extract_page_elements의 return 값에서 제공
    "page_metadata": {}, # page_metadata() 함수
    "page_summary": {}, # 아직 구현 x
    "images": [], # crop_image_and_table
    "images_summary": [], # create_image_summary_data_batches
    "tables": [], # crop_image_and_table
    "tables_summary": {}, # create_image_summary_data_batches
    "texts": {}, # crop_image_and_table
    "texts_summary": {} # create_image_summary_data_batches
}

In [65]:
## 문서를 배치 단위로 추출
def split_pdf(state: GraphState):
    """
    입력 PDF를 여러 개의 작은 PDF 파일로 분할합니다.
    :param state: GraphState 객체, PDF 파일 경로와 배치 크기 정보를 포함
    :return: 분할된 PDF 파일 경로 목록을 포함한 GraphState 객체
    """
    filepath = state["filepath"]
    batch_size = state["batch_size"]

    input_pdf = pymupdf.open(filepath)
    num_pages = len(input_pdf)
    print(f"총 페이지 수: {num_pages}")

    ret = []
    for start_page in range(0, num_pages, batch_size):
        end_page = min(start_page + batch_size, num_pages) - 1

        input_file_basename = os.path.splitext(filepath)[0]
        output_file = f"{input_file_basename}_{start_page:04d}_{end_page:04d}.pdf"
        print(f"분할 PDF 생성: {output_file}")

        with pymupdf.open() as output_pdf:
            output_pdf.insert_pdf(input_pdf, from_page=start_page, to_page=end_page)
            output_pdf.save(output_file)
            ret.append(output_file)

    input_pdf.close()
    return GraphState(split_filepaths=ret)

In [66]:
def extract_page_metadata(state: GraphState) -> GraphState:
    filepath = state["filepath"]          # ✅ 반드시 state 사용 (하드코딩 금지)
    doc = pymupdf.open(filepath)
    metadata = {}
    for i, page in enumerate(doc, start=1):
        rect = page.rect
        metadata[i] = {"size": [int(rect.width), int(rect.height)]}
    doc.close()
    state["page_metadata"] = metadata
    return state


state_out = extract_page_metadata(state)
state.update(state_out)
state["page_metadata"] # 왼쪽 키는 page 번호, 옆은 valye

{}

In [67]:
import os

def extract_start_end_page(filename: str):
    """
    파일명에 포함된 '_0000_0009' 같은 패턴에서 시작/끝 페이지 추출
    :param filename: JSON 또는 PDF 파일 경로
    :return: (start_page, end_page) 튜플 (int)
    """
    # 파일명만 추출
    basename = os.path.splitext(os.path.basename(filename))[0]
    
    # 언더스코어(_) 기준으로 분리
    parts = basename.split("_")
    
    try:
        # 뒤에서 두 개가 start, end 페이지 번호
        start = int(parts[-2])
        end = int(parts[-1])
    except Exception:
        start, end = 0, 0  # 실패 시 기본값
    
    return start, end

In [68]:
import glob
import os

def extract_page_elements(state: GraphState):
    # JSON 파일 폴더 지정 (PDF가 있는 폴더 기반으로)
    ## 이 위치 반영하여 처리 : /home/user/rag/energy_statistic/2024산업부문에너지온실가스실태조사및통계분석보고서_0170_0179.json
    json_dir = "/home/user/langchain-study/2024_수요보고서"

    print(json_dir)
    
    # 폴더 안의 모든 JSON 파일 탐색
    json_files = sorted(glob.glob(os.path.join(json_dir, "*.json")))
    state["analyzed_files"] = json_files

    page_elements = dict()
    element_id = 0

    for json_file in json_files:
        start_page, _ = extract_start_end_page(json_file)

        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)

        for element in data["elements"]:
            original_page = int(element["page"])
            relative_page = start_page + original_page - 1

            if relative_page not in page_elements:
                page_elements[relative_page] = []

            element["id"] = element_id
            element_id += 1
            element["page"] = relative_page

            page_elements[relative_page].append(element)

    return GraphState(page_elements=page_elements, analyzed_files=json_files)

In [69]:
def extract_start_end_page(filename: str) -> tuple[int, int]:
    basename = os.path.splitext(os.path.basename(filename))[0]
    parts = basename.split("_")
    try:
        start = int(parts[-2]); end = int(parts[-1])
    except Exception:
        start, end = 0, 0
    return start, end

state_out = extract_page_elements(state)
state.update(state_out)
state["page_elements"] # 왼쪽 키는 page 번호, 옆은 valye

/home/user/langchain-study/2024_수요보고서


{}

In [70]:
def extract_tag_elements_per_page(state: GraphState) -> GraphState:
    """
    page_elements를 이미지, 표, 텍스트로 분류
    """
    page_elements = state["page_elements"]
    parsed_page_elements = {}

    for key, page_element in page_elements.items():
        image_elements = []
        table_elements = []
        text_elements = []

        for element in page_element:
            if element["category"] in ["figure", "chart"]:
                image_elements.append(element)
            elif element["category"] == "table":
                table_elements.append(element)
            else:
                text_elements.append(element)

        parsed_page_elements[key] = {
            "image_elements": image_elements,
            "table_elements": table_elements,
            "text_elements": text_elements,
            "elements": page_element,  # 원본 포함
        }

    return GraphState(page_elements=parsed_page_elements)

In [71]:
state_out = extract_tag_elements_per_page(state)
state.update(state_out)

In [72]:
state

{'filepath': '',
 'filetype': 'pdf',
 'page_numbers': [],
 'batch_size': 4,
 'split_filepaths': [],
 'analyzed_files': [],
 'page_elements': {},
 'page_metadata': {},
 'page_summary': {},
 'images': [],
 'images_summary': [],
 'tables': [],
 'tables_summary': {},
 'texts': {},
 'texts_summary': {}}

In [73]:
import os, json, pymupdf
from PIL import Image, ImageDraw

def crop_image_and_table(state: GraphState, output_dir="output_images", save_debug=True) -> GraphState:
    os.makedirs(output_dir, exist_ok=True)
    
    pdf_file = state["filepath"]
    json_files = state["analyzed_files"]
    
    images, tables, texts = {}, {}, {}
    
    with pymupdf.open(pdf_file) as doc:
        for json_file in json_files:
            with open(json_file, "r", encoding="utf-8") as f:
                data = json.load(f)
            
            # --- 메타데이터 (페이지 크기) ---
            meta_pages = {p.get("page"): (p.get("width"), p.get("height"))
                          for p in data.get("metadata", {}).get("pages", [])}

            filename = os.path.basename(json_file)
            if "_" in filename:
                parts = filename.replace(".json", "").split("_")
                try:
                    start_page = int(parts[-2]) + 1
                except:
                    start_page = 1
            else:
                start_page = 1
            
            for element in data.get("elements", []):
                category = element.get("category", "unknown")
                coords = element.get("bounding_box", [])
                page_num = int(element.get("page", 1))
                relative_page = start_page + page_num - 1
                element_id = element.get("id", "na")

                # -------------------------
                # 텍스트 저장
                # -------------------------
                if category in ["text", "header", "footer", "paragraph", 
                                "heading1", "heading2", "list", "index"]:
                    txt = element.get("text", "").strip()
                    if txt:
                        texts.setdefault(relative_page, []).append({
                            "page": relative_page,
                            "id": element_id,
                            "content": txt
                        })
                    continue

                # 좌표/페이지 유효성 검사
                if not coords or relative_page - 1 >= len(doc):
                    continue

                page = doc[relative_page - 1]
                pix = page.get_pixmap(dpi=300)
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                img_w, img_h = img.size

                meta_w, meta_h = meta_pages.get(page_num, (page.rect.width, page.rect.height))
                scale_x, scale_y = img_w / meta_w, img_h / meta_h

                xs = [c["x"] for c in coords]
                ys = [c["y"] for c in coords]
                left, right = int(min(xs) * scale_x), int(max(xs) * scale_x)
                top, bottom = int(min(ys) * scale_y), int(max(ys) * scale_y)

                crop_left, crop_top = max(0, left), max(0, top)
                crop_right, crop_bottom = min(img_w, right), min(img_h, bottom)

                if crop_right <= crop_left or crop_bottom <= crop_top:
                    continue

                fname = f"page{relative_page}_id{element_id}_{category}.png"
                fpath = os.path.join(output_dir, fname)

                cropped_img = img.crop((crop_left, crop_top, crop_right, crop_bottom))
                cropped_img.save(fpath)

                if save_debug:
                    debug_img = img.copy()
                    draw = ImageDraw.Draw(debug_img)
                    colors = {"table": "blue", "figure": "green", "chart": "orange"}
                    color = colors.get(category, "red")
                    draw.rectangle([left, top, right, bottom], outline=color, width=2)
                    draw.text((crop_left, max(0, crop_top - 20)), f"{category}-{element_id}", fill="red")
                    debug_path = os.path.join(output_dir, f"DEBUG_{fname}")
                    debug_img.save(debug_path)

                # -------------------------
                # 표 처리 (통합 구조)
                # -------------------------
                if category == "table":
                    table_text = element.get("text", "").strip()
                    md_table = ""
                    if table_text:
                        rows = [row.strip() for row in table_text.split("\n") if row.strip()]
                        md_table_list = []
                        for i, row in enumerate(rows):
                            cells = [c for c in row.split() if c]
                            md_row = "| " + " | ".join(cells) + " |"
                            md_table_list.append(md_row)
                            if i == 0:
                                md_table_list.append("| " + " | ".join(["---"] * len(cells)) + " |")
                        md_table = "\n".join(md_table_list)
                    
                    tables.setdefault(relative_page, []).append({
                        "page": relative_page,
                        "id": element_id,
                        "markdown": md_table,
                        "image_path": fpath,
                        "summary": None,      # 후처리 요약 예정
                        "category": "table"
                    })

                # -------------------------
                # 순수 이미지 처리 (figure, chart)
                # -------------------------
                elif category in ["figure", "chart"]:
                    images.setdefault(relative_page, []).append({
                        "page": relative_page,
                        "id": element_id,
                        "path": fpath,
                        "category": category
                    })
    
    return GraphState(images=images, tables=tables, texts=texts)

In [74]:
import re
import os

_RANGE_PDF_RE = re.compile(r"_\d{4}_\d{4}\.pdf$", re.IGNORECASE)

def is_main_pdf(filename: str) -> bool:
    """page-range 분할 PDF(예: _0000_0009.pdf)는 제외하고, 나머지 PDF만 True"""
    if not filename.lower().endswith(".pdf"):
        return False
    if _RANGE_PDF_RE.search(filename):
        return False
    return True


def find_jsons_for_pdf(pdf_path: str, search_roots: list[str]) -> list[str]:
    """
    pdf의 base 이름을 기준으로 JSON 파일을 폭넓게 탐색:
      - <base>.json
      - <base>_*.json        (배치 분할: _0000_0009.json 등)
      - <some_dir>/<base>/*.json  (동명 폴더 안)
    """
    base = os.path.splitext(os.path.basename(pdf_path))[0]
    base_lower = base.lower()
    hits = []

    for root in search_roots:
        if not os.path.isdir(root):
            continue

        # 1) 재귀적으로 *.json 훑어서 base 또는 base_ 로 시작하면 채택
        for r, _, files in os.walk(root):
            for f in files:
                if not f.lower().endswith(".json"):
                    continue
                name_noext = os.path.splitext(f)[0].lower()
                if (name_noext == base_lower) or name_noext.startswith(base_lower + "_"):
                    hits.append(os.path.join(r, f))

        # 2) <root>/<base>/*.json (동명 폴더)
        base_dir = os.path.join(root, base)
        if os.path.isdir(base_dir):
            for f in os.listdir(base_dir):
                if f.lower().endswith(".json"):
                    hits.append(os.path.join(base_dir, f))

    return sorted(set(hits))


In [75]:
import os
from glob import glob
import re
import unicodedata

def find_jsons_for_pdf(pdf_path: str, search_roots: list[str]) -> list[str]:
    """
    PDF 파일에 대응하는 분석 JSON을 여러 후보 폴더에서 찾아 반환.
    """
    base = os.path.splitext(os.path.basename(pdf_path))[0]
    base_lower = base.lower()
    hits = []

    for root in search_roots:
        if not os.path.isdir(root):
            continue

        # 재귀적으로 *.json 탐색
        for r, _, files in os.walk(root):
            for f in files:
                if not f.lower().endswith(".json"):
                    continue
                name_noext = os.path.splitext(f)[0].lower()
                if (name_noext == base_lower) or name_noext.startswith(base_lower + "_"):
                    hits.append(os.path.join(r, f))

        # 동명 폴더 탐색
        base_dir = os.path.join(root, base)
        if os.path.isdir(base_dir):
            for f in os.listdir(base_dir):
                if f.lower().endswith(".json"):
                    hits.append(os.path.join(base_dir, f))

    return sorted(set(hits))


def resolve_korean_dir(root: str, target_basename: str) -> str | None:
    """
    root 안에서 target_basename과 같은(또는 정규화하면 같은) 디렉토리를 찾아 절대경로 반환.
    """
    target_nfc = unicodedata.normalize("NFC", target_basename)
    target_nfd = unicodedata.normalize("NFD", target_basename)
    for d in os.listdir(root):
        bn = os.path.basename(d)
        if (
            bn == target_basename
            or unicodedata.normalize("NFC", bn) == target_nfc
            or unicodedata.normalize("NFD", bn) == target_nfd
        ):
            p = os.path.join(root, bn)
            if os.path.isdir(p):
                return p
    for d in os.listdir(root):
        bn = os.path.basename(d)
        if bn.startswith("2019") and ("수요보고서" in bn or "수요보고서" in bn):
            p = os.path.join(root, bn)
            if os.path.isdir(p):
                return p
    return None


if __name__ == "__main__":
    root = "/home/user/langchain-study"
    target = "2024_수요보고서"
    pdf_dir = resolve_korean_dir(root, target)

    if not pdf_dir or not os.path.isdir(pdf_dir):
        raise FileNotFoundError(f"폴더를 찾지 못했습니다: {root}/{target}")

    visuals_root = "/home/user/langchain-study/2024_visuals_output"
    os.makedirs(visuals_root, exist_ok=True)

    _RANGE_PDF_RE = re.compile(r"_\d{4}_\d{4}\.pdf$", re.IGNORECASE)
    def is_main_pdf(filename: str) -> bool:
        return filename.lower().endswith(".pdf") and not _RANGE_PDF_RE.search(filename)

    json_search_roots = [pdf_dir]

    # 메인 PDF만 선별
    pdf_list = [f for f in sorted(os.listdir(pdf_dir)) if is_main_pdf(f)]
    print(f"🔎 찾은 메인 PDF: {len(pdf_list)}개 @ {pdf_dir}")
    print(f"📋 PDF 목록: {pdf_list[:5]}{'...' if len(pdf_list) > 5 else ''}")

    # 🔥 각 PDF의 결과를 독립적으로 저장
    all_results = {}
    skipped_pdfs = []
    
    for i, fname in enumerate(pdf_list, 1):
        pdf_path = os.path.join(pdf_dir, fname)
        base = os.path.splitext(fname)[0]
        output_dir = os.path.join(visuals_root, base)
        os.makedirs(output_dir, exist_ok=True)

        print(f"\n[{i}/{len(pdf_list)}] 📄 처리 시작: {fname}")

        # 🔥 각 PDF마다 완전히 독립적인 state 생성
        state = {
            "filepath": pdf_path,
            "analyzed_files": [],
            "page_elements": {},
            "tables": {},
            "texts": {},
            "images": {}
        }

        # JSON 찾기
        jsons = find_jsons_for_pdf(pdf_path, json_search_roots)
        if jsons:
            state["analyzed_files"] = jsons
            print(f"   🧩 JSON {len(jsons)}개 연결")
        else:
            print(f"   ⚠️ JSON 없음 → extract_page_elements로 생성 시도")
            try:
                result = extract_page_elements(state)
                # 🔥 중요: update 대신 개별 키만 복사
                if result.get("analyzed_files"):
                    state["analyzed_files"] = result["analyzed_files"]
                if result.get("page_elements"):
                    state["page_elements"] = result["page_elements"]
                
                if not state.get("analyzed_files"):
                    print(f"   ❗ analyzed_files 비어있음 → 스킵")
                    skipped_pdfs.append((fname, "analyzed_files 없음"))
                    continue
            except Exception as e:
                print(f"   ❌ extract_page_elements 실패: {e}")
                skipped_pdfs.append((fname, f"추출 실패: {e}"))
                continue

        # 각 단계별 처리
        try:
            # extract_tag_elements_per_page 처리
            result1 = extract_tag_elements_per_page(state)
            if result1.get("tables"):
                state["tables"] = result1["tables"]
            if result1.get("texts"):
                state["texts"] = result1["texts"]
            if result1.get("images"):
                state["images"] = result1["images"]
            
            print(f"   🔧 태그 요소 추출 완료")
            
            # crop_image_and_table 처리
            result2 = crop_image_and_table(state, output_dir=output_dir, save_debug=True)
            
            # 🔥 중요: 반환된 결과를 state에 병합 (덮어쓰지 않고)
            if result2.get("tables"):
                state["tables"].update(result2["tables"])
            if result2.get("texts"):
                state["texts"].update(result2["texts"])
            if result2.get("images"):
                state["images"].update(result2["images"])
            
            print(f"   🖼️ 이미지/표 크롭 완료")
            
            # 🔥 결과를 딕셔너리에 저장
            all_results[base] = state
            
            # 간단한 통계 출력
            table_count = sum(len(v) for v in state.get("tables", {}).values())
            text_count = sum(len(v) for v in state.get("texts", {}).values())
            image_count = sum(len(v) for v in state.get("images", {}).values())
            print(f"   ✅ 완료: 표 {table_count}개, 텍스트 {text_count}개, 이미지 {image_count}개")
            
        except Exception as e:
            import traceback
            print(f"   ❌ 처리 중 에러 발생: {e}")
            print(f"   상세: {traceback.format_exc()}")
            skipped_pdfs.append((fname, f"처리 실패: {e}"))
            continue

    # 🔥 전체 결과 요약
    print("\n" + "="*60)
    print("=== 전체 PDF 처리 결과 요약 ===")
    print("="*60)
    print(f"✅ 성공: {len(all_results)}개")
    print(f"❌ 스킵: {len(skipped_pdfs)}개")
    
    if skipped_pdfs:
        print("\n⚠️ 스킵된 PDF 목록:")
        for fname, reason in skipped_pdfs:
            print(f"  - {fname}: {reason}")
    
    if all_results:
        print("\n📊 처리 완료된 PDF 상세:")
        for pdf_name, state in all_results.items():
            table_count = sum(len(v) for v in state.get("tables", {}).values())
            text_count = sum(len(v) for v in state.get("texts", {}).values())
            image_count = sum(len(v) for v in state.get("images", {}).values())
            print(f"\n📄 {pdf_name}:")
            print(f"  - 표: {table_count}개")
            print(f"  - 텍스트: {text_count}개")
            print(f"  - 이미지: {image_count}개")
            print(f"  - filepath: {state.get('filepath', 'N/A')}")
    
    print(f"\n✨ 총 {len(pdf_list)}개 중 {len(all_results)}개 PDF 처리 완료")
    
    # 🔥 옵션: 전체 결과를 JSON으로 저장
    # import json
    # summary_path = os.path.join(visuals_root, "processing_summary.json")
    # with open(summary_path, "w", encoding="utf-8") as f:
    #     summary = {
    #         k: {
    #             "filepath": v["filepath"],
    #             "table_count": sum(len(t) for t in v.get("tables", {}).values()),
    #             "text_count": sum(len(t) for t in v.get("texts", {}).values()),
    #             "image_count": sum(len(i) for i in v.get("images", {}).values())
    #         }
    #         for k, v in all_results.items()
    #     }
    #     json.dump(summary, f, ensure_ascii=False, indent=2)
    # print(f"\n💾 요약 파일 저장: {summary_path}")

🔎 찾은 메인 PDF: 12개 @ /home/user/langchain-study/2024_수요보고서
📋 PDF 목록: ['2024년10월전력수요예측전망.pdf', '2024년11월전력수요예측전망.pdf', '2024년12월전력수요예측전망.pdf', '2024년1월전력수요예측전망_v1.pdf', '2024년2월전력수요예측전망.pdf']...

[1/12] 📄 처리 시작: 2024년10월전력수요예측전망.pdf
   🧩 JSON 1개 연결
   🔧 태그 요소 추출 완료
   🖼️ 이미지/표 크롭 완료
   ✅ 완료: 표 3개, 텍스트 46개, 이미지 5개

[2/12] 📄 처리 시작: 2024년11월전력수요예측전망.pdf
   🧩 JSON 1개 연결
   🔧 태그 요소 추출 완료
   🖼️ 이미지/표 크롭 완료
   ✅ 완료: 표 3개, 텍스트 45개, 이미지 9개

[3/12] 📄 처리 시작: 2024년12월전력수요예측전망.pdf
   🧩 JSON 1개 연결
   🔧 태그 요소 추출 완료
   🖼️ 이미지/표 크롭 완료
   ✅ 완료: 표 3개, 텍스트 43개, 이미지 7개

[4/12] 📄 처리 시작: 2024년1월전력수요예측전망_v1.pdf
   🧩 JSON 1개 연결
   🔧 태그 요소 추출 완료
   🖼️ 이미지/표 크롭 완료
   ✅ 완료: 표 5개, 텍스트 72개, 이미지 6개

[5/12] 📄 처리 시작: 2024년2월전력수요예측전망.pdf
   🧩 JSON 1개 연결
   🔧 태그 요소 추출 완료
   🖼️ 이미지/표 크롭 완료
   ✅ 완료: 표 5개, 텍스트 70개, 이미지 6개

[6/12] 📄 처리 시작: 2024년3월전력수ᄋ

In [15]:
# doc_state = all_results["2020년11월전력수요예측전망"]
# doc_state.pop("tables", None)

## 캡처된 이미지 서머리 확인

In [76]:
import os
import base64
import time
import json
from typing import Iterable, List, Dict, Any
from openai import OpenAI

client = OpenAI()


def parse_filename(fname: str) -> Dict[str, Any] | None:
    """
    Expected filename pattern: page{n}_id{m}_{category}.png
    Example: page2_id10_chart.png
    Excludes debug/caption images.
    """
    if not fname.lower().endswith(".png"):
        return None
    if "DEBUG" in fname.upper():
        return None
    if "caption" in fname.lower():
        return None
    try:
        base = fname[:-4]  # Remove .png
        parts = base.split("_")   # ['page2','id10','chart']
        page = int(parts[0].replace("page", ""))
        elem_id = int(parts[1].replace("id", ""))
        category = parts[2].lower() if len(parts) > 2 else "unknown"
        return {"page": page, "id": elem_id, "category": category}
    except Exception:
        return None


def _iter_image_files(folders: Iterable[str]):
    """
    Recursively traverses the given root folder(s) and generates metadata for images to summarize.
    The document identifier doc_slug uses 'the folder name where each file is located (last directory name of root)'.
    """
    for folder in folders:
        if not os.path.isdir(folder):
            continue
        # Recursively traverse all subfolders
        for root, _, files in os.walk(folder):
            for fname in files:
                meta = parse_filename(fname)
                if meta is None:
                    continue
                path = os.path.join(root, fname)
                # Document identifier: folder name where the current file is located
                doc_slug = os.path.basename(root)
                yield {
                    "doc": doc_slug,
                    "page": meta["page"],
                    "id": meta["id"],
                    "category": meta["category"],
                    "path": path,
                }


def load_images_with_auto_meta(folders_or_folder) -> List[Dict[str, Any]]:
    """
    Takes a single path (str) or list of paths and returns all image metadata as a sorted list.
    """
    if isinstance(folders_or_folder, str):
        folders = [folders_or_folder]
    else:
        folders = list(folders_or_folder)
    items = list(_iter_image_files(folders))
    return sorted(items, key=lambda x: (x["doc"], x["page"], x["id"]))


def create_image_summary_data_batches(
    state: Dict[str, Any],
    folders_or_folder,
    delay: float = 1.0,
    limit: int | None = None,
    checkpoint_file: str = "state_checkpoint.json"
) -> Dict[str, Any]:
    """
    Summarizes recursively scanned images using OpenAI and accumulates them in state["images_summary"].
    Prevents duplicates by UID (doc/page/id). Saves progress to checkpoint file.
    """
    images_meta = load_images_with_auto_meta(folders_or_folder)
    print(f"🔎 Number of scanned images: {len(images_meta)}")
    summaries: List[Dict[str, Any]] = []

    # Set of already processed UIDs
    existing_uids = {s["uid"] for s in state.get("images_summary", [])}

    for idx, meta in enumerate(images_meta):
        if limit is not None and limit >= 0 and idx >= limit:
            break

        page_num = meta["page"]
        uid = f"{meta['doc']}_p{page_num}_id{meta['id']}"

        if uid in existing_uids:
            print(f"⚠️ {uid} already processed → skipping")
            continue

        # Page context (use if available)
        text_context = ""
        texts_summary = state.get("texts_summary") or state.get("text_summary") or {}
        if isinstance(texts_summary, dict):
            page_info = texts_summary.get(page_num)
            if isinstance(page_info, str):
                text_context = page_info
            elif isinstance(page_info, dict):
                text_context = page_info.get("summary", "") or ""

        img_path = meta["path"]
        with open(img_path, "rb") as f:
            img_base64 = base64.b64encode(f.read()).decode()

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system",
                     "content": (
                         "You are a highly respected analyst specializing in electricity demand analysis. "
                         "Respond in English. "
                         "When analyzing tables, carefully examine numerical data and provide specific details without omitting any numbers. "
                         "When analyzing graphs, describe the axes, values, and trends in detail. "
                         "Never omit numerical values. Carefully check axis information and base your explanation on it. "
                         "For both tables and graphs, if context is provided, analyze them together with the context."
                     )},
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": (
                                    f"[Document] {meta['doc']}\n"
                                    f"[Page] {page_num}\n\n"
                                    f"Page summary (context):\n{text_context}\n\n"
                                    f"Please interpret this {meta['category']} image along with the context, "
                                    f"providing a detailed summary of numerical values, axes, and trends."
                                )
                            },
                            {"type": "image_url",
                             "image_url": {"url": f"data:image/png;base64,{img_base64}"}}
                        ]
                    }
                ],
                max_tokens=1000,
                temperature=0.0
            )
        except Exception as e:
            print(f"❌ Error occurred (uid={uid}): {e}")
            break

        content = response.choices[0].message.content
        summary = content.strip() if isinstance(content, str) else ""

        rec = {
            "uid": uid,
            "doc": meta["doc"],
            "page": page_num,
            "id": meta["id"],
            "category": meta["category"],
            "path": img_path,
            "summary": summary
        }
        summaries.append(rec)
        print(f"✅ {uid} summary completed")

        # Checkpoint
        state.setdefault("images_summary", []).append(rec)
        with open(checkpoint_file, "w", encoding="utf-8") as f:
            json.dump(state, f, ensure_ascii=False, indent=2)

        time.sleep(delay)

    return state


# ===================== Execution Example =====================
if __name__ == "__main__":
    # Example state (if only running summaries, images/texts structure is optional)
    state = {
        "images_summary": [],     # Accumulates existing summaries. Initialize to [] to start fresh
        "texts_summary": {}       # Add page context if available (works without it)
    }

    # ✅ Actual image root (recursively scans subfolders)
    image_root = "/home/user/langchain-study/2024_visuals_output"
    # To process multiple roots at once, use a list like this:
    # image_root = ["/home/user/langchain-study/2019_visuals_output",
    #               "/home/user/langchain-study/2020_visuals_output"]

    # (Optional) Delete checkpoint file for complete reset
    # if os.path.exists("state_checkpoint.json"):
    #     os.remove("state_checkpoint.json")
    # state["images_summary"] = []

    # Execute
    state = create_image_summary_data_batches(
        state,
        image_root,
        delay=4,
        limit=None
    )

    # Report
    imgs = load_images_with_auto_meta(image_root)
    print("📊 Total scanned images:", len(imgs))
    print("📑 Generated summaries:", len(state.get("images_summary", [])))

    # Sample output
    for rec in state.get("images_summary", [])[:5]:
        print("➡️", rec["uid"], "|", rec["path"])

🔎 Number of scanned images: 122
✅ 2024년10월전력수요예측전망_p2_id10 summary completed
✅ 2024년10월전력수요예측전망_p2_id13 summary completed
✅ 2024년10월전력수요예측전망_p3_id19 summary completed
✅ 2024년10월전력수요예측전망_p3_id23 summary completed
✅ 2024년10월전력수요예측전망_p4_id32 summary completed
✅ 2024년10월전력수요예측전망_p5_id36 summary completed
✅ 2024년10월전력수요예측전망_p6_id44 summary completed
✅ 2024년10월전력수요예측전망_p8_id57 summary completed
✅ 2024년11월전력수요예측전망_p2_id9 summary completed
✅ 2024년11월전력수요예측전망_p2_id12 summary completed
✅ 2024년11월전력수요예측전망_p3_id18 summary completed
✅ 2024년11월전력수요예측전망_p3_id22 summary completed
✅ 2024년11월전력수요예측전망_p4_id31 summary completed
✅ 2024년11월전력수요예측전망_p4_id32 summary completed
✅ 2024년11월전력수요예측전망_p4_id36 summary completed
✅ 2024년11월전력수요예측전망_p5_

In [77]:
state["images_summary"]

save_path = "/home/user/langchain-study/2024_images_summary_.json"
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(state["images_summary"], f, ensure_ascii=False, indent=2)

In [78]:
import re

def group_images_by_month(state):
    """state['images_summary']를 문서명에서 '2019 01' 같은 키로 묶어 반환."""
    imgs = state.get("images_summary", [])
    grouped = {}

    for i in imgs:
        doc = i.get("doc", "")
        # 문서명에서 연도, 월 추출 (예: 2019년12월 → "2019 12")
        m = re.search(r"(\d{4})\s*년\s*(\d{1,2})\s*월", doc)
        if m:
            key = f"{m.group(1)} {int(m.group(2)):02d}"
        else:
            # 패턴 못 찾으면 doc 이름 그대로
            key = doc

        # 필요한 필드만 추출
        item = {
            "doc": i.get("doc"),
            "category": i.get("category"),
            "summary": i.get("summary")
        }

        grouped.setdefault(key, []).append(item)

    return grouped

In [79]:
grouped = group_images_by_month(state)

# state에 저장하고 싶다면
state["grouped_by_month"] = grouped

# 보기 좋게 출력
for k, v in grouped.items():
    print(f"{k}: {len(v)}개")
    for item in v[:2]:  # 앞 2개만 미리보기
        print("  ", item["category"], "→", item["summary"], "...")


2024년10월전력수요예측전망: 8개
   chart → The chart displays electricity demand forecasts from 2020 to 2024, with the y-axis representing demand in megawatts (MW) ranging from 65,000 to 100,000 MW, and the x-axis representing months from 1 to 12.

### Key Observations:

- **2024 Forecast (Red Line):**
  - Peaks at 97,115 MW in the middle of the year.
  - Lowest demand is 69,899 MW around May.
  - Notable values include 89,231 MW in January and 93,246 MW in September.

- **2023 Data (Blue Line):**
  - Peaks at 94,509 MW.
  - Lowest demand is 71,199 MW around April.
  - Other significant points are 85,400 MW in October and 91,556 MW in December.

- **2022 Data (Pink Line):**
  - Peaks at 92,990 MW.
  - Lowest demand is 72,292 MW around April.

- **2021 Data (Black Line):**
  - Peaks at 86,855 MW.
  - Lowest demand is 66,699 MW around May.

- **2020 Data (Green Line):**
  - Peaks at 85,132 MW.
  - Lowest demand is 68,454 MW around November.

### Trends:

- There is a general trend 

In [80]:
# text 확인해서 보여주기 - 해당코드는 절대 수정 x, 하더라도 한번만 클릭하자
state_summary = state["images_summary"]
# state_summary

In [81]:
state_summary

[{'uid': '2024년10월전력수요예측전망_p2_id10',
  'doc': '2024년10월전력수요예측전망',
  'page': 2,
  'id': 10,
  'category': 'chart',
  'path': '/home/user/langchain-study/2024_visuals_output/2024년10월전력수요예측전망/page2_id10_chart.png',
  'summary': 'The chart displays electricity demand forecasts from 2020 to 2024, with the y-axis representing demand in megawatts (MW) ranging from 65,000 to 100,000 MW, and the x-axis representing months from 1 to 12.\n\n### Key Observations:\n\n- **2024 Forecast (Red Line):**\n  - Peaks at 97,115 MW in the middle of the year.\n  - Lowest demand is 69,899 MW around May.\n  - Notable values include 89,231 MW in January and 93,246 MW in September.\n\n- **2023 Data (Blue Line):**\n  - Peaks at 94,509 MW.\n  - Lowest demand is 71,199 MW around April.\n  - Other significant points are 85,400 MW in October and 91,556 MW in December.\n\n- **2022 Data (Pink Line):**\n  - Peaks at 92,990 MW.\n  - Lowest demand is 72,292 MW around April

## 두 결과를 결합한다.

In [82]:
def merge_summaries_into_state(all_results: dict, summaries: list) -> dict:
    """
    Vision API로 생성된 summary 데이터를 all_results의 state에 병합
    
    Args:
        all_results: {pdf_basename: state} 형태의 딕셔너리
        summaries: [{uid, doc, page, id, category, path, summary}, ...] 리스트
    
    Returns:
        병합된 all_results
    """
    # uid를 키로 하는 lookup 딕셔너리 생성
    summary_lookup = {item['uid']: item for item in summaries}
    
    matched_count = 0
    missing_count = 0
    
    for pdf_name, state in all_results.items():
        # 표(tables) 병합
        for page, tables in state.get("tables", {}).items():
            for table in tables:
                # uid 생성: {pdf_name}_p{page}_id{id}
                uid = f"{pdf_name}_p{page}_id{table['id']}"
                if uid in summary_lookup:
                    table['description'] = summary_lookup[uid]['summary']
                    matched_count += 1
                else:
                    print(f"⚠️ 매칭 실패 (표): {uid}")
                    missing_count += 1
        
        # 이미지(images) 병합
        for page, images in state.get("images", {}).items():
            for image in images:
                uid = f"{pdf_name}_p{page}_id{image['id']}"
                if uid in summary_lookup:
                    image['description'] = summary_lookup[uid]['summary']
                    matched_count += 1
                else:
                    print(f"⚠️ 매칭 실패 (이미지): {uid}")
                    missing_count += 1
    
    print(f"\n✅ 병합 완료: {matched_count}개 성공, {missing_count}개 실패")
    
    return all_results


def create_flat_visual_elements(all_results: dict) -> list:
    """
    all_results에서 표와 이미지를 평탄화하여 리스트로 반환
    텍스트와 함께 사용하기 쉬운 형태로 변환
    
    Returns:
        [{
            'doc': pdf_name,
            'page': page_num,
            'id': element_id,
            'category': 'table' | 'chart' | 'figure',
            'path': image_path,
            'description': summary_text
        }, ...]
    """
    visual_elements = []
    
    for pdf_name, state in all_results.items():
        # 표 추가
        for page, tables in state.get("tables", {}).items():
            for table in tables:
                visual_elements.append({
                    'doc': pdf_name,
                    'page': page,
                    'id': table['id'],
                    'category': 'table',
                    'path': table.get('image_path', ''),
                    'description': table.get('description', '')
                })
        
        # 이미지 추가
        for page, images in state.get("images", {}).items():
            for image in images:
                visual_elements.append({
                    'doc': pdf_name,
                    'page': page,
                    'id': image['id'],
                    'category': image['category'],
                    'path': image.get('path', ''),
                    'description': image.get('description', '')
                })
    
    return visual_elements


def create_flat_text_elements(all_results: dict) -> list:
    """
    all_results에서 텍스트를 평탄화하여 리스트로 반환
    
    Returns:
        [{
            'doc': pdf_name,
            'page': page_num,
            'id': element_id,
            'content': text_content
        }, ...]
    """
    text_elements = []
    
    for pdf_name, state in all_results.items():
        for page, texts in state.get("texts", {}).items():
            for text in texts:
                text_elements.append({
                    'doc': pdf_name,
                    'page': page,
                    'id': text['id'],
                    'content': text['content']
                })
    
    return text_elements


def merge_texts_and_visuals_by_page(all_results: dict) -> dict:
    """
    페이지별로 텍스트와 비주얼 요소를 병합
    
    Returns:
        {
            'pdf_name': {
                page_num: {
                    'texts': [...],
                    'visuals': [...]
                }
            }
        }
    """
    merged = {}
    
    for pdf_name, state in all_results.items():
        merged[pdf_name] = {}
        
        # 모든 페이지 번호 수집
        all_pages = set()
        all_pages.update(state.get("texts", {}).keys())
        all_pages.update(state.get("tables", {}).keys())
        all_pages.update(state.get("images", {}).keys())
        
        for page in sorted(all_pages):
            page_data = {
                'texts': [],
                'visuals': []
            }
            
            # 텍스트 추가
            if page in state.get("texts", {}):
                for text in state["texts"][page]:
                    page_data['texts'].append({
                        'id': text['id'],
                        'content': text['content']
                    })
            
            # 표 추가
            if page in state.get("tables", {}):
                for table in state["tables"][page]:
                    page_data['visuals'].append({
                        'id': table['id'],
                        'category': 'table',
                        'path': table.get('image_path', ''),
                        'description': table.get('description', '')
                    })
            
            # 이미지 추가
            if page in state.get("images", {}):
                for image in state["images"][page]:
                    page_data['visuals'].append({
                        'id': image['id'],
                        'category': image['category'],
                        'path': image.get('path', ''),
                        'description': image.get('description', '')
                    })
            
            # id 순으로 정렬
            page_data['texts'].sort(key=lambda x: x['id'])
            page_data['visuals'].sort(key=lambda x: x['id'])
            
            merged[pdf_name][page] = page_data
    
    return merged


# ============================================================
# 사용 예시
# ============================================================

if __name__ == "__main__":
    # 1. Summary 데이터 병합
    summaries = [
        {'uid': '2019년01월전력수요예측전망_p2_id10', 'summary': '차트 설명...'},
        {'uid': '2019년01월전력수요예측전망_p2_id13', 'summary': '표 설명...'},
        # ... 더 많은 summary 데이터
    ]
    
    # all_results에 summary 병합
    all_results = merge_summaries_into_state(all_results, summaries)
    
    # 2-A. 평탄화된 리스트 생성 (RAG 용도)
    visual_elements = create_flat_visual_elements(all_results)
    text_elements = create_flat_text_elements(all_results)
    
    print(f"✅ 비주얼 요소: {len(visual_elements)}개")
    print(f"✅ 텍스트 요소: {len(text_elements)}개")
    
    # 2-B. 페이지별 병합 (문서 재구성 용도)
    page_merged = merge_texts_and_visuals_by_page(all_results)
    
    print(f"\n📄 페이지별 병합 완료:")
    for pdf_name, pages in page_merged.items():
        print(f"  {pdf_name}: {len(pages)}페이지")
        for page_num, data in list(pages.items())[:2]:  # 첫 2페이지만 출력
            print(f"    - 페이지 {page_num}: 텍스트 {len(data['texts'])}개, 비주얼 {len(data['visuals'])}개")
    
    # 3. JSON으로 저장 (옵션)
    import json
    
    # 평탄화 버전 저장
    with open("visual_elements.json", "w", encoding="utf-8") as f:
        json.dump(visual_elements, f, ensure_ascii=False, indent=2)
    
    with open("text_elements.json", "w", encoding="utf-8") as f:
        json.dump(text_elements, f, ensure_ascii=False, indent=2)
    
    # 페이지별 병합 버전 저장
    with open("page_merged.json", "w", encoding="utf-8") as f:
        json.dump(page_merged, f, ensure_ascii=False, indent=2)
    
    print("\n💾 JSON 파일 저장 완료")

⚠️ 매칭 실패 (표): 2024년10월전력수요예측전망_p2_id13
⚠️ 매칭 실패 (표): 2024년10월전력수요예측전망_p3_id19
⚠️ 매칭 실패 (표): 2024년10월전력수요예측전망_p3_id23
⚠️ 매칭 실패 (이미지): 2024년10월전력수요예측전망_p2_id10
⚠️ 매칭 실패 (이미지): 2024년10월전력수요예측전망_p4_id32
⚠️ 매칭 실패 (이미지): 2024년10월전력수요예측전망_p5_id36
⚠️ 매칭 실패 (이미지): 2024년10월전력수요예측전망_p6_id44
⚠️ 매칭 실패 (이미지): 2024년10월전력수요예측전망_p8_id57
⚠️ 매칭 실패 (표): 2024년11월전력수요예측전망_p2_id12
⚠️ 매칭 실패 (표): 2024년11월전력수요예측전망_p3_id18
⚠️ 매칭 실패 (표): 2024년11월전력수요예측전망_p3_id22
⚠️ 매칭 실패 (이미지): 2024년11월전력수요예측전망_p2_id9
⚠️ 매칭 실패 (이미지): 2024년11월전력수요예측전망_p4_id31
⚠️ 매칭 실패 (이미지): 2024년11월전력수요예측전망_p4_id32
⚠️ 매칭 실패 (이미지): 2024년11월전력수요예측전망_p4_id36
⚠️ 매칭 실패 (이미지): 2024년11월전력수요예측전망_p5_id41
⚠️ 매칭 실패 (이미지): 2024년11월전력수요예측전망_p5_id42
⚠️ 매칭 실패 (이미지): 2024년11워

In [83]:
# all_results의 각 PDF별 표/이미지 개수 확인
for pdf_name, state in all_results.items():
    table_count = sum(len(tables) for tables in state.get("tables", {}).values())
    image_count = sum(len(images) for images in state.get("images", {}).values())
    print(f"{pdf_name}: 표 {table_count}개, 이미지 {image_count}개")

2024년10월전력수요예측전망: 표 3개, 이미지 5개
2024년11월전력수요예측전망: 표 3개, 이미지 9개
2024년12월전력수요예측전망: 표 3개, 이미지 7개
2024년1월전력수요예측전망_v1: 표 5개, 이미지 6개
2024년2월전력수요예측전망: 표 5개, 이미지 6개
2024년3월전력수요예측전망: 표 3개, 이미지 8개
2024년4월전력수요예측전망: 표 3개, 이미지 8개
2024년5월전력수요예측전망: 표 3개, 이미지 9개
2024년6월전력수요예측전망: 표 3개, 이미지 6개
2024년7월전력수요예측전망: 표 3개, 이미지 6개
2024년8월전력수요예측전망: 표 3개, 이미지 6개
2024년9월전력수요예측전망: 표 3개, 이미지 6개


In [84]:
all_results = merge_summaries_into_state(all_results, state_summary)
all_results


✅ 병합 완료: 122개 성공, 0개 실패


{'2024년10월전력수요예측전망': {'filepath': '/home/user/langchain-study/2024_수요보고서/2024년10월전력수요예측전망.pdf',
  'analyzed_files': ['/home/user/langchain-study/2024_수요보고서/2024년10월전력수요예측전망/2024년10월전력수요예측전망_0000_0007.json'],
  'page_elements': {},
  'tables': {2: [{'page': 2,
     'id': 13,
     'markdown': '| 구 | 분 | `20.10월 | `21.10월 | `22.10월 | `23.10월 | `24.10월 |\n| --- | --- | --- | --- | --- | --- | --- |\n| 최대수요 | (㎿) | (전년대비 | 증가율, | %) | 68,454 | (-6.9%) | 75,698 | (10.6%) | 72,544 | (-4.2%) | 70,985 | (-2.1%) | 77,000 | (8.5%) |\n| 평균수요 | (㎿) | (전년대비 | 증가율, | %) | 56,627 | (-3.0%) | 59,775 | (5.6%) | 59,032 | (-1.2%) | 57,731 | (-2.2%) | 60,200 | (4.3%) |',
     'image_path': '/home/user/langchain-study/2024_visuals_output/2024년10월전력수요예측전망/page2_id13_table.png',
     'summary': None,
     'category': 'table',
     'description': 'The table presents electricity demand forecasts for October from 2020 to 2024, f

In [86]:
for pdf_name, state in all_results.items():
    for page, tables in state.get("tables", {}).items():
        for table in tables:
            table.pop('markdown', None)
            table.pop('summary', None)  # 이것도 None이니 제거

In [108]:
# 코드 수정 필요 - 필요없는 key 값 제거 
"""
주피터 노트북에서 바로 실행 가능한 PDF 파싱 구조화 코드 (일반화 버전)
- 특정 월(1월/2월) 하드코딩 제거
- 본문에서 "YYYY년 M월 주별 최대전력"을 감지해서 서브섹션 생성
- 이어지는 테이블을 해당 서브섹션에 순서대로 매칭
- "과거 전력수요 실적그래프" 바로 아래에 차트 연결
- "최근 5개년 실적 및 전망" 안정 파싱(연/최대/평균/증감률)
"""

import json
import re
from typing import Dict, List, Any, Optional
from datetime import datetime
from pathlib import Path
import pandas as pd


class PowerDemandReportParser:
    """전력수요예측 보고서 PDF 파싱 결과를 구조화하는 클래스"""

    def __init__(self):
        self.section_markers = {
            'primary': [r'^\d+\.\s+', r'^\[붙임\]'],
            'secondary': [r'^○\s+', r'^Ϛ\s+']
        }

    # -------------------- 유틸 --------------------
    def _clean_bullet(self, text: str) -> str:
        """머리표/특수문자 제거 (○, Ϛ, •, -, ※ 등)"""
        return re.sub(r'^[\s\u25CB\u221A\u03DA\u2022○Ϛ\-\•※]+[\s]*', '', text).strip()

    def _is_bullet_or_note_line(self, text: str) -> bool:
        """머리표(○/Ϛ/•/-) 또는 주석(※)으로 시작하는 라인인지"""
        return bool(re.match(r'^[\s]*(?:○|Ϛ|•|\-+|※)\s+', text))


    # -------------------- 메인 --------------------
    def parse_document(self, raw_data: Dict) -> Dict:
        """메인 파싱 함수"""
        document = {
            "document_info": self._extract_document_info(raw_data),
            "sections": self._organize_sections(raw_data),
            "summary": self._generate_summary(raw_data),
            "metadata": self._extract_metadata(raw_data)
        }
        return document

    # -------------------- 문서 정보 --------------------
    def _extract_document_info(self, raw_data: Dict) -> Dict:
        """문서 기본 정보 추출"""
        info = {
            "title": "월간 전력수요 전망",
            "date": "",
            "author": "",
            "contact": "",
            "filepath": raw_data.get('filepath', '')
        }

        texts = raw_data.get('texts', {})

        if 1 in texts:
            for item in texts[1]:
                content = item['content']
                if re.search(r'\d{4}\.\s*\d{1,2}\.', content):
                    info['date'] = content.strip()
                elif '중앙전력관제센터' in content:
                    info['author'] = content.replace('\n', ' ').strip()
                elif '작성자' in content:
                    info['contact'] = content.strip()

        if 'filepath' in raw_data:
            filename = Path(raw_data['filepath']).stem
            info['title'] = filename.replace('_', ' ').strip()

        return info

    # -------------------- 섹션 구성 --------------------
    def _organize_sections(self, raw_data: Dict) -> List[Dict]:
        """텍스트를 논리적 섹션으로 구성"""
        sections = []
        texts = raw_data.get('texts', {})
        images = raw_data.get('images', {})
        tables = raw_data.get('tables', {})

        # 섹션별로 정리 (페이지 기본값은 추후 텍스트에서 감지된 페이지를 사용)
        section_map = {
            1: {"title": "기상전망", "page": 2, "subsections": []},
            2: {"title": "과거 전력수요 추이", "page": 2, "subsections": []},
            3: {"title": "전력수요 전망 결과", "page": 3, "subsections": []},
            4: {"title": "기상전망 상세", "page": 4, "subsections": []}
        }

        # === 페이지별 텍스트 스캔 ===
        # 1) 페이지 2: 기상전망 / 과거 전력수요
        if 2 in texts:
            current_section = None
            for item in texts[2]:
                content_raw = item['content']
                content = content_raw.strip()

                if '1. 기상전망' in content:
                    current_section = 1
                    section_map[1]['page'] = 2
                    continue
                elif '2. 과거 전력수요' in content:
                    current_section = 2
                    section_map[2]['page'] = 2
                    continue

                # ---------------- 기상전망 ----------------
                if current_section == 1:
                    # 섹션 제목(예: "1월 기온 전망", "1월 강수량 전망") → 새 서브섹션 생성
                    if ('기온 전망' in content) or ('강수량 전망' in content):
                        section_map[1]['subsections'].append({
                            "subtitle": content,
                            "content": "",
                            "page": 2,
                            "related_data": None
                        })
                        continue

                    # 머리표/주석 라인 → 항상 직전 서브섹션의 content에 누적
                    if self._is_bullet_or_note_line(content) or content.startswith('※'):
                        if section_map[1]['subsections']:
                            prev = section_map[1]['subsections'][-1].get('content', '')
                            add  = self._clean_bullet(content.lstrip('※').strip())
                            section_map[1]['subsections'][-1]['content'] = (prev + '\n' + add).strip() if prev else add
                        else:
                            # 제목이 먼저 안 나왔는데 내용이 먼저 온 경우: 임시 요약 서브섹션에 수용
                            section_map[1]['subsections'].append({
                                "subtitle": "기상전망 요약",
                                "content": self._clean_bullet(content.lstrip('※').strip()),
                                "page": 2,
                                "related_data": None
                            })
                        continue

                    # 그 외 일반 줄: 직전 서브섹션 content에 이어붙임(이상치 방지용)
                    if section_map[1]['subsections']:
                        prev = section_map[1]['subsections'][-1].get('content', '')
                        section_map[1]['subsections'][-1]['content'] = (prev + '\n' + content).strip() if prev else content
                    else:
                        section_map[1]['subsections'].append({
                            "subtitle": "기상전망 요약",
                            "content": content,
                            "page": 2,
                            "related_data": None
                        })

                # ---------------- 과거 전력수요 추이 ----------------
                elif current_section == 2:
                    # 실적그래프/최근5개년 제목만 서브섹션으로 생성 (머리표 라인은 content로만)
                    if '과거년도 전력수요 실적그래프' in content:
                        section_map[2]['subsections'].append({
                            "subtitle": "과거 전력수요 실적그래프",
                            "content": "",
                            "page": 2,
                            "related_data": None
                        })
                    elif '최근 5개년 실적 및 전망' in content:
                        section_map[2]['subsections'].append({
                            "subtitle": "최근 5개년 실적 및 전망",
                            "content": "",
                            "page": 2,
                            "related_data": None
                        })
                    else:
                        # 머리표/주석 등은 마지막 서브섹션 content에 누적
                        if self._is_bullet_or_note_line(content) or content.startswith('※'):
                            if section_map[2]['subsections']:
                                prev = section_map[2]['subsections'][-1].get('content', '')
                                add  = self._clean_bullet(content.lstrip('※').strip())
                                section_map[2]['subsections'][-1]['content'] = (prev + '\n' + add).strip() if prev else add


        # 2) 페이지 3: 전력수요 전망 결과 (연·월 주별 섹션 제목을 동적 탐지)
        #    예: "2019년 1월 주별 최대전력", "2020년 12월 주별 최대전력"
        month_subtitles: List[Dict[str, Any]] = []  # 테이블 매칭용 큐
        if 3 in texts:
            for item in texts[3]:
                content = item['content'].strip()
                # 최대/평균 부하 요약
                if '최대부하 예측' in content:
                    section_map[3]['page'] = 3
                    section_map[3]['subsections'].append({
                        "subtitle": "최대부하 예측",
                        "content": content.split(':', 1)[1].strip() if ':' in content else content,
                        "page": 3,
                        "related_data": None
                    })
                elif '평균부하 예측' in content:
                    section_map[3]['page'] = 3
                    section_map[3]['subsections'].append({
                        "subtitle": "평균부하 예측",
                        "content": content.split(':', 1)[1].strip() if ':' in content else content,
                        "page": 3,
                        "related_data": None
                    })
                elif any(k in content for k in ['ARIMA', '지수평활', '회귀분석', 'SEFS']):
                    if section_map[3]['subsections']:
                        section_map[3]['subsections'][-1]['methodology'] = content
                # "YYYY년 M월 주별 최대전력" 제목 감지
                m = re.search(r'(\d{4})년\s*(\d{1,2})월\s*주별\s*최대전력', content)
                if m:
                    year, month = m.group(1), m.group(2)
                    subtitle = f"{year}년 {month}월 주별 최대전력"
                    subsec = {
                        "subtitle": subtitle,
                        "content": "",
                        "page": 3,
                        "related_data": None
                    }
                    section_map[3]['subsections'].append(subsec)
                    month_subtitles.append(subsec)

        # === 이미지/테이블 연결 ===
        # 페이지 2 - 차트: "실적그래프" 바로 아래로
        if 2 in images:
            for img in images[2]:
                target = None
                for sub in section_map[2]['subsections']:
                    if '실적그래프' in sub.get('subtitle', ''):
                        target = sub
                        break
                if target is None:
                    target = {
                        "subtitle": "과거 전력수요 실적그래프",
                        "content": "",
                        "page": 2,
                        "related_data": None
                    }
                    section_map[2]['subsections'].append(target)

                target['related_data'] = {
                    'type': 'chart',
                    'data': self._parse_chart_data(img)
                }

        # 페이지 2 - 테이블: "최근 5개년 실적 및 전망" 바로 아래로
        if 2 in tables:
            for tbl in tables[2]:
                target = None
                for sub in section_map[2]['subsections']:
                    if '최근 5개년' in sub.get('subtitle', ''):
                        target = sub
                        break
                if target is None:
                    target = {
                        "subtitle": "최근 5개년 실적 및 전망",
                        "content": "",
                        "page": 2,
                        "related_data": None
                    }
                    section_map[2]['subsections'].append(target)

                target['related_data'] = {
                    'type': 'table',
                    'data': self._parse_table_data(tbl)
                }

        # 페이지 3 - 테이블: 감지된 "YYYY년 M월 주별 최대전력" 서브섹션 순서대로 매칭
        if 3 in tables:
            idx = 0
            for tbl in tables[3]:
                if idx < len(month_subtitles):
                    month_subtitles[idx]['related_data'] = {
                        'type': 'table',
                        'data': self._parse_table_data(tbl)
                    }
                    idx += 1
                else:
                    # 남는 테이블은 일반 "주별 최대전력"으로 추가
                    section_map[3]['subsections'].append({
                        "subtitle": "주별 최대전력",
                        "content": "",
                        "page": 3,
                        "related_data": {
                            'type': 'table',
                            'data': self._parse_table_data(tbl)
                        }
                    })

        # 페이지 4 - 붙임/기상 상세
        if 4 in texts:
            for item in texts[4]:
                content = item['content'].strip()
                if '[붙임]' in content:
                    section_map[4]['page'] = 4
                    continue
                elif '1월은' in content or '2월 전반' in content or '상층 한기' in content:
                    section_map[4]['subsections'].append({
                        "subtitle": "기상 패턴 전망",
                        "content": content,
                        "page": 4,
                        "related_data": None
                    })

        if 4 in tables:
            for tbl in tables[4]:
                section_map[4]['subsections'].append({
                    "subtitle": "주별 기상 상세",
                    "content": "",
                    "page": 4,
                    "related_data": {
                        'type': 'table',
                        'data': self._parse_table_data(tbl)
                    }
                })

        # 섹션 맵을 리스트로 변환
        for idx, section_data in section_map.items():
            if section_data['subsections']:
                sections.append({
                    "section_id": idx,
                    "title": section_data['title'],
                    "page": section_data['page'],
                    "subsections": section_data['subsections']
                })

        return sections

    # -------------------- 차트 파싱 --------------------
    def _parse_chart_data(self, chart_info: Dict) -> Dict:
        """차트 정보 파싱 (data_points 미사용)"""
        description = chart_info.get('description', '')

        data = {
            'description': description,
            'image_path': chart_info.get('path', ''),
            'key_points': [],
            'data_points': [],   # ← 비워둠
            'value_unit': None   # 단위도 굳이 쓸 필요 없으면 None
        }

        # 핵심 포인트(서술형 키워드만 유지하고 싶으면 유지, 이마저도 불필요하면 아래 2줄 삭제)
        if '최고치' in description:
            data['key_points'].append('여름철(7-8월) 최고 수요')
        if '최저치' in description:
            data['key_points'].append('봄/가을 최저 수요')

        return data


    # -------------------- 테이블 파싱 --------------------
    def _parse_table_data(self, table_info: Dict) -> Dict:
        """테이블 정보 파싱 및 구조화"""
        description = table_info.get('description', '')

        data = {
            'description': description,
            'image_path': table_info.get('image_path', ''),
            'structured_data': self._extract_structured_data(description)
        }

        return data

    def _extract_structured_data(self, description: str) -> Dict:
        """테이블 설명에서 구조화된 데이터 추출"""
        structured = {
            'headers': [],
            'rows': [],
            'data_type': None,
            'fields': []
        }

        desc = description.replace('\u200b', ' ').strip()

        # ---------- 주차별 (주차/기간/값) ----------
        # 기간과 값이 분리되어 있어도 병합 가능
        if ('주차' in desc or '주별' in desc or re.search(r'\d+주\s*\(', desc)) and ('만 kW' in desc or '만kW' in desc):
            structured['data_type'] = 'weekly'
            structured['headers'] = ['주차', '기간', '최대부하(만kW)']
            structured['fields'] = ['week', 'period', 'value']

            # 1) 기간: "1주 (12/31 ~ 1/6)"
            period_map = {}
            for w, p in re.findall(r'(\d+주(?:차)?)\s*\(\s*([^)]+?)\s*\)', desc):
                wk = w.replace('차', '')
                period_map[wk] = p.strip()

            # 2) 값: "1주차: 8,600만 kW"
            for w, val in re.findall(r'(\d+주(?:차)?)\s*[:：]\s*(\d+,?\d*)\s*만?\s*kW', desc):
                wk = w.replace('차', '')
                structured['rows'].append({
                    'week': w,
                    'period': period_map.get(wk, ''),
                    'value': int(val.replace(',', ''))
                })

            return structured

        # ---------- 연도별 (최근 5개년 실적 및 전망) ----------
        if ('최근 5개년' in desc or ('최대부하' in desc and '평균부하' in desc)) and re.search(r'\d{4}년', desc):
            structured['data_type'] = 'yearly'
            structured['headers'] = ['연도', '최대부하(만kW)', '증감률(%)', '평균부하(만kW)', '증감률(%)']
            structured['fields'] = ['year', 'max_load', 'max_rate', 'avg_load', 'avg_rate']

            # 텍스트가 "최대부하" 구간과 "평균부하" 구간을 구분해서 나열하는 패턴 대응
            # 최대부하 파트
            max_part = ''
            avg_part = ''
            # "최대부하" 이후 ~ "평균부하" 직전
            m1 = re.search(r'최대부하[^:]*:\s*(.*?)(?:\n\s*-\s*평균부하|평균부하|$)', desc, flags=re.S)
            if m1:
                max_part = m1.group(1)
            # "평균부하" 이후
            m2 = re.search(r'평균부하[^:]*:\s*(.*)$', desc, flags=re.S)
            if m2:
                avg_part = m2.group(1)

            max_block = re.findall(r'-\s*(\d{4})년(?:\s*\d{1,2}월)?\s*:\s*([\d,]+)\s*\(\s*([+-]?\d+\.?\d*%?)\s*\)', max_part)
            avg_block = re.findall(r'-\s*(\d{4})년(?:\s*\d{1,2}월)?\s*:\s*([\d,]+)\s*\(\s*([+-]?\d+\.?\d*%?)\s*\)', avg_part)

            # 백업 패턴 (구분 실패 시 텍스트 전체에서 탐색)
            if not max_block:
                max_block = re.findall(r'-\s*(\d{4})년(?:\s*\d{1,2}월)?\s*:\s*([\d,]+)\s*\(\s*([+-]?\d+\.?\d*%?)\s*\)', desc)
            if not avg_block and '평균부하' in desc:
                # 평균부하 키워드 뒤에서만 추출 시도
                tail = desc.split('평균부하', 1)[-1]
                avg_block = re.findall(r'-\s*(\d{4})년(?:\s*\d{1,2}월)?\s*:\s*([\d,]+)\s*\(\s*([+-]?\d+\.?\d*%?)\s*\)', tail)

            # 병합
            row_map: Dict[str, Dict[str, Any]] = {}
            order: List[str] = []

            for y, v, r in max_block:
                year = y.strip()
                order.append(year)
                row_map[year] = {
                    'year': year,
                    'max_load': int(v.replace(',', '')),
                    'max_rate': r.strip(),
                    'avg_load': '',
                    'avg_rate': ''
                }

            for y, v, r in avg_block:
                year = y.strip()
                if year not in row_map:
                    order.append(year)
                    row_map[year] = {
                        'year': year,
                        'max_load': '',
                        'max_rate': '',
                        'avg_load': int(v.replace(',', '')),
                        'avg_rate': r.strip()
                    }
                else:
                    row_map[year]['avg_load'] = int(v.replace(',', ''))
                    row_map[year]['avg_rate'] = r.strip()

            for y in order:
                structured['rows'].append(row_map[y])

            return structured

        # ---------- 기상(주별 기온/강수량 범위) ----------
        if '기온' in desc or '강수량' in desc:
            structured['data_type'] = 'weather'
            structured['headers'] = ['주차', '기온범위(°C)', '강수량(mm)']
            structured['fields'] = ['week', 'temp_range', 'precip_range']

            for i in range(1, 6):
                row = {'week': f'{i}주차'}
                t = re.search(rf'{i}주[^-：:]*(-?\d+\.?\d*)\s*°C\s*~\s*(-?\d+\.?\d*)\s*°C', desc)
                p = re.search(rf'{i}주[^-：:]*([\d\.]+)\s*mm\s*~\s*([\d\.]+)\s*mm', desc)
                if t:
                    row['temp_range'] = f'{t.group(1)}°C ~ {t.group(2)}°C'
                if p:
                    row['precip_range'] = f'{p.group(1)}mm ~ {p.group(2)}mm'
                if len(row) > 1:
                    structured['rows'].append(row)

            return structured

        return structured

    # -------------------- 요약 --------------------
    def _generate_summary(self, raw_data: Dict) -> Dict:
        """문서 요약 정보 생성"""
        summary = {
            'key_predictions': {},
            'trend_analysis': {},
            'weather_impact': {}
        }

        texts = raw_data.get('texts', {})

        all_text = ""
        for page_texts in texts.values():
            for item in page_texts:
                all_text += item['content'] + " "

        # 주요 예측값 (예시 패턴)
        mmax = re.search(r'최대부하\s*예측\s*[:：]\s*([\d,]+)\s*만?kW', all_text)
        if mmax:
            summary['key_predictions']['max_load'] = mmax.group(1) + "만kW"
        mavg = re.search(r'평균부하\s*예측\s*[:：]\s*([\d,]+)\s*만?kW', all_text)
        if mavg:
            summary['key_predictions']['avg_load'] = mavg.group(1) + "만kW"

        if re.search(r'[-+]\s*0\.6%', all_text) or '-0.6%' in all_text:
            summary['trend_analysis']['max_load_change'] = "-0.6%"
        if re.search(r'[-+]\s*0\.3%', all_text) or '-0.3%' in all_text:
            summary['trend_analysis']['avg_load_change'] = "-0.3%"

        if '평년과 비슷' in all_text:
            summary['weather_impact']['temperature'] = '평년과 비슷'
        if '기온 변동성' in all_text:
            summary['weather_impact']['volatility'] = '크겠음'

        return summary

    # -------------------- 메타데이터 --------------------
    def _extract_metadata(self, raw_data: Dict) -> Dict:
        """메타데이터 추출"""
        return {
            'total_pages': len(raw_data.get('texts', {})),
            'charts_count': sum(len(imgs) for imgs in raw_data.get('images', {}).values()),
            'tables_count': sum(len(tbls) for tbls in raw_data.get('tables', {}).values()),
            'processing_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }


# ============= 주피터 노트북에서 사용할 함수들 =============
def process_pdf_data(raw_data: Dict) -> Dict:
    parser = PowerDemandReportParser()
    structured_doc = parser.parse_document(raw_data)

    print("✅ 문서 구조화 완료!")
    print(f"📄 제목: {structured_doc['document_info']['title']}")
    print(f"📅 작성일: {structured_doc['document_info']['date']}")
    print(f"📊 섹션 수: {len(structured_doc['sections'])}")

    return structured_doc


def show_summary(doc: Dict):
    print("\n" + "=" * 60)
    print("📊 전력수요예측 보고서 요약")
    print("=" * 60)

    info = doc['document_info']
    print(f"\n📄 문서 정보:")
    print(f"  - 제목: {info['title']}")
    print(f"  - 작성일: {info['date']}")
    print(f"  - 작성자: {info['author']}")
    print(f"  - 연락처: {info['contact']}")

    predictions = doc['summary']['key_predictions']
    if predictions:
        print(f"\n🎯 핵심 예측:")
        if 'max_load' in predictions:
            print(f"  - 최대부하: {predictions['max_load']}")
        if 'avg_load' in predictions:
            print(f"  - 평균부하: {predictions['avg_load']}")

    trend = doc['summary']['trend_analysis']
    if trend:
        print(f"\n📈 전년대비 변화:")
        print(f"  - 최대부하: {trend.get('max_load_change', 'N/A')}")
        print(f"  - 평균부하: {trend.get('avg_load_change', 'N/A')}")

    print(f"\n📑 문서 구조:")
    for section in doc['sections']:
        print(f"  {section['section_id']}. {section['title']}")
        for sub in section['subsections'][:2]:
            if sub['subtitle']:
                print(f"     - {sub['subtitle']}")

    meta = doc['metadata']
    print(f"\n📊 메타데이터:")
    print(f"  - 총 페이지: {meta['total_pages']}")
    print(f"  - 차트 수: {meta['charts_count']}")
    print(f"  - 테이블 수: {meta['tables_count']}")


def extract_tables_to_df(doc: Dict) -> Dict[str, pd.DataFrame]:
    dataframes = {}

    for section in doc['sections']:
        for subsection in section['subsections']:
            if subsection.get('related_data') and subsection['related_data']['type'] == 'table':
                sd = subsection['related_data']['data']['structured_data']
                if sd.get('rows'):
                    fields = sd.get('fields')
                    if fields:
                        df = pd.DataFrame([{k: row.get(k, '') for k in fields} for row in sd['rows']])
                    else:
                        df = pd.DataFrame(sd['rows'])

                    df_name = f"{section['title']}_{subsection['subtitle']}"
                    dataframes[df_name] = df
                    print(f"\n📊 {df_name}")
                    print(df)

    return dataframes


def get_weekly_forecast(doc: Dict, year: Optional[int] = None, month: Optional[int] = None) -> pd.DataFrame:
    """
    주별 예측 데이터를 DataFrame으로 반환
    - year, month가 주어지면 해당 (YYYY, M) 섹션만
    - 아무것도 주어지지 않으면 문서의 모든 '주별' 섹션 합쳐서 반환
    """
    weekly_data = []

    def month_in_subtitle(subtitle: str) -> Optional[int]:
        m = re.search(r'(\d{4})년\s*(\d{1,2})월', subtitle)
        if m:
            return int(m.group(2))
        return None

    def year_in_subtitle(subtitle: str) -> Optional[int]:
        m = re.search(r'(\d{4})년', subtitle)
        if m:
            return int(m.group(1))
        return None

    for section in doc['sections']:
        for subsection in section['subsections']:
            st = subsection.get('subtitle', '')
            if '주별' in st and subsection.get('related_data'):
                sd = subsection['related_data']['data']['structured_data']
                if sd.get('data_type') == 'weekly':
                    y_ok = True if year is None else (year_in_subtitle(st) == year)
                    m_ok = True if month is None else (month_in_subtitle(st) == month)
                    if y_ok and m_ok:
                        weekly_data.extend(sd['rows'])

    if weekly_data:
        cols = ['week', 'period', 'value']
        df = pd.DataFrame([{k: row.get(k, '') for k in cols} for row in weekly_data])
        print("\n📈 주별 전력수요 예측")
        return df
    else:
        print("⚠️ 해당 조건의 주별 데이터를 찾을 수 없습니다.")
        return pd.DataFrame()


def save_to_files(doc: Dict, base_name: str = "output"):
    json_file = f"{base_name}_structured.json"
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(doc, f, ensure_ascii=False, indent=2)
    print(f"✅ JSON 파일 저장: {json_file}")

    md_content = create_markdown_report(doc)
    md_file = f"{base_name}_report.md"
    with open(md_file, 'w', encoding='utf-8') as f:
        f.write(md_content)
    print(f"✅ 마크다운 파일 저장: {md_file}")


def create_markdown_report(doc: Dict) -> str:
    md = []

    info = doc['document_info']
    md.append(f"# {info['title']}\n")
    md.append(f"- **작성일**: {info['date']}")
    md.append(f"- **작성자**: {info['author']}")
    md.append(f"- **연락처**: {info['contact']}\n")

    summary = doc['summary']
    if summary['key_predictions']:
        md.append("## 📊 핵심 예측\n")
        for key, value in summary['key_predictions'].items():
            md.append(f"- **{key}**: {value}")
        md.append("")

    for section in doc['sections']:
        md.append(f"\n## {section['section_id']}. {section['title']}\n")

        for subsection in section['subsections']:
            if subsection['subtitle']:
                md.append(f"### {subsection['subtitle']}\n")

            if subsection['content']:
                md.append(subsection['content'].strip() + "\n")

            if subsection.get('related_data'):
                rd = subsection['related_data']
                if rd['type'] == 'table':
                    sd = rd['data']['structured_data']
                    if sd.get('rows') and sd.get('headers'):
                        md.append("| " + " | ".join(sd['headers']) + " |")
                        md.append("|" + "---|" * len(sd['headers']))
                        fields = sd.get('fields', [])
                        if fields:
                            for row in sd['rows']:
                                values = [str(row.get(k, '')) for k in fields]
                                md.append("| " + " | ".join(values) + " |")
                        else:
                            for row in sd['rows']:
                                values = [str(v) for v in row.values()]
                                md.append("| " + " | ".join(values) + " |")
                        md.append("")
                elif rd['type'] == 'chart':
                    img_path = rd['data'].get('image_path', '')
                    if img_path:
                        md.append(f"![차트]({img_path})\n")

    return "\n".join(md)


# ============= 메인 실행 예제 =============
print("📚 전력수요예측 보고서 파서(일반화) 로드 완료!")
print("사용법:")
print("1. structured_doc = process_pdf_data(your_json_data)")
print("2. show_summary(structured_doc)")
print("3. dfs = extract_tables_to_df(structured_doc)")
print("4. weekly_df = get_weekly_forecast(structured_doc, year=2019, month=1)  # or None")
print("5. save_to_files(structured_doc, 'output_filename')")


📚 전력수요예측 보고서 파서(일반화) 로드 완료!
사용법:
1. structured_doc = process_pdf_data(your_json_data)
2. show_summary(structured_doc)
3. dfs = extract_tables_to_df(structured_doc)
4. weekly_df = get_weekly_forecast(structured_doc, year=2019, month=1)  # or None
5. save_to_files(structured_doc, 'output_filename')


In [105]:
all_results

{'2024년10월전력수요예측전망': {'filepath': '/home/user/langchain-study/2024_수요보고서/2024년10월전력수요예측전망.pdf',
  'analyzed_files': ['/home/user/langchain-study/2024_수요보고서/2024년10월전력수요예측전망/2024년10월전력수요예측전망_0000_0007.json'],
  'page_elements': {},
  'tables': {2: [{'page': 2,
     'id': 13,
     'image_path': '/home/user/langchain-study/2024_visuals_output/2024년10월전력수요예측전망/page2_id13_table.png',
     'category': 'table',
     'description': 'The table presents electricity demand forecasts for October from 2020 to 2024, focusing on maximum and average demand in megawatts (MW), along with year-over-year percentage changes.\n\n### Maximum Demand (MW)\n- **2020.10월**: 68,454 MW, a decrease of 6.9% from the previous year.\n- **2021.10월**: 75,698 MW, an increase of 10.6%.\n- **2022.10월**: 72,544 MW, a decrease of 4.2%.\n- **2023.10월**: 70,985 MW, a decrease of 2.1%.\n- **2024.10월**: Forecasted at 77,000 MW, an increase of 8.

In [109]:
# 해당 코드도 날자 맞춰서 변경하기
structured_doc_202101 = process_pdf_data(all_results["2024년1월전력수요예측전망_v1"])
structured_doc_202102 = process_pdf_data(all_results["2024년2월전력수요예측전망"])
structured_doc_202103 = process_pdf_data(all_results["2024년3월전력수요예측전망"])
structured_doc_202104 = process_pdf_data(all_results["2024년4월전력수요예측전망"])
structured_doc_202105 = process_pdf_data(all_results["2024년5월전력수요예측전망"])
structured_doc_202106 = process_pdf_data(all_results["2024년6월전력수요예측전망"])
structured_doc_202107 = process_pdf_data(all_results["2024년7월전력수요예측전망"])
structured_doc_202108 = process_pdf_data(all_results["2024년8월전력수요예측전망"])
structured_doc_202109 = process_pdf_data(all_results["2024년9월전력수요예측전망"])
structured_doc_202110 = process_pdf_data(all_results["2024년10월전력수요예측전망"])
structured_doc_202111 = process_pdf_data(all_results["2024년11월전력수요예측전망"])
structured_doc_202112 = process_pdf_data(all_results["2024년12월전력수요예측전망"])

✅ 문서 구조화 완료!
📄 제목: 2024년1월전력수요예측전망 v1
📅 작성일: 2024. 12.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2024년2월전력수요예측전망
📅 작성일: 2024. 1.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2024년3월전력수요예측전망
📅 작성일: 2024. 2.
📊 섹션 수: 3
✅ 문서 구조화 완료!
📄 제목: 2024년4월전력수요예측전망
📅 작성일: 2024. 3.
📊 섹션 수: 3
✅ 문서 구조화 완료!
📄 제목: 2024년5월전력수요예측전망
📅 작성일: 2024. 4.
📊 섹션 수: 3
✅ 문서 구조화 완료!
📄 제목: 2024년6월전력수요예측전망
📅 작성일: 2024. 5.
📊 섹션 수: 3
✅ 문서 구조화 완료!
📄 제목: 2024년7월전력수요예측전망
📅 작성일: 2024. 7.
📊 섹션 수: 3
✅ 문서 구조화 완료!
📄 제목: 2024년8월전력수요예측전망
📅 작성일: 2024. 7.
📊 섹션 수: 3
✅ 문서 구조화 완료!
📄 제목: 2024년9월전력수요예측전망
📅 작성일: 2024. 8.
📊 섹션 수: 3
✅ 문서 구조화 완료!
📄 제목: 2024년10월전력수요예측전망
📅 작성일: 2024. 9.
📊 섹션 수: 3
✅ 문서 구조화 완료!
📄 제목: 2024년11월전력수요예측전망
📅 작성일: 2024. 10.
📊 섹션 수: 3
✅ 문서 구조화 완료!
📄 제목: 2024년12월전력수요예측전망
📅 작성일: 2024. 11.
📊 섹션 수: 3


In [111]:
structured_doc_202112

{'document_info': {'title': '2024년12월전력수요예측전망',
  'date': '2024. 11.',
  'author': '중앙전력관제센터 수 요 예 측 팀',
  'contact': '작성자 : 수요예측팀 차장 손흥구(☏8802)\n대리 송민호(☏8803)\n대리 최요환(☏8809)',
  'filepath': '/home/user/langchain-study/2024_수요보고서/2024년12월전력수요예측전망.pdf'},
 'sections': [{'section_id': 1,
   'title': '기상전망',
   'page': 2,
   'subsections': [{'subtitle': '기상전망 요약',
     'content': '’24.12~’25.1월 기상전망\n12월 기온은 평년과 비슷하겠고, 강수량은 평년보다 대체로 적겠음\n- 기 온 : 평년(0.5~1.7℃) 보다 낮음 30%, 비슷 50%, 높음 20%\n- 강수량 : 평년(19.8~28.6㎜) 보다 적음 40%, 비슷 40%, 많음 20%\n1월 기온은 평년과 비슷하겠고, 강수량은 평년보다 대체로 적겠음\n- 기 온 : 평년(-1.5~-0.3℃) 보다 낮음 20%, 비슷 50%, 높음 30%\n- 강수량 : 평년(17.4~26.8㎜) 보다 적음 40%, 비슷 40%, 많음 20%',
     'page': 2,
     'related_data': None}]},
  {'section_id': 2,
   'title': '과거 전력수요 추이',
   'page': 2,
   'subsections': [{'subtitle': '과거 전력수요 실적그래프',
     'content': '',
     'page': 2,
     'related_data': {'type': 'chart',
      'data': {'description': 'The chart displays electricity 

In [112]:
import json
def save_structured_doc(structured_doc, filename):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(structured_doc, f, ensure_ascii=False, indent=2)

# 
save_structured_doc(structured_doc_202101, "structured_doc_202401.json")
save_structured_doc(structured_doc_202102, "structured_doc_202402.json")
save_structured_doc(structured_doc_202103, "structured_doc_202403.json")
save_structured_doc(structured_doc_202104, "structured_doc_202404.json")
save_structured_doc(structured_doc_202105, "structured_doc_202405.json")
save_structured_doc(structured_doc_202106, "structured_doc_202406.json")
save_structured_doc(structured_doc_202107, "structured_doc_202407.json")
save_structured_doc(structured_doc_202108, "structured_doc_202408.json")
save_structured_doc(structured_doc_202109, "structured_doc_202409.json")
save_structured_doc(structured_doc_202110, "structured_doc_202410.json")
save_structured_doc(structured_doc_202111, "structured_doc_202411.json")
save_structured_doc(structured_doc_202112, "structured_doc_202412.json")

### 2019

In [293]:
structured_doc_201901 = process_pdf_data(all_results["2019년01월전력수요예측전망"])
structured_doc_201902 = process_pdf_data(all_results["2019년02월전력수요예측전망"])
structured_doc_201903 = process_pdf_data(all_results["2019년03월전력수요예측전망"])
structured_doc_201904 = process_pdf_data(all_results["2019년04월전력수요예측전망"])
structured_doc_201905 = process_pdf_data(all_results["2019년05월전력수요예측전망_v1"])
structured_doc_201906 = process_pdf_data(all_results["2019년06월전력수요예측전망_v1"])
structured_doc_201907 = process_pdf_data(all_results["2019년07월전력수요예측전망_v2 (2)"])
structured_doc_201908 = process_pdf_data(all_results["2019년08월전력수요예측전망_v4"])
structured_doc_201909 = process_pdf_data(all_results["2019년09월전력수요예측전망_v1"])
structured_doc_201910 = process_pdf_data(all_results["2019년10월전력수요예측전망_v1"])
structured_doc_201911 = process_pdf_data(all_results["2019년11월전력수요예측전망_v1"])
structured_doc_201912 = process_pdf_data(all_results["2019년12월전력수요예측전망_v1"])

✅ 문서 구조화 완료!
📄 제목: 2019년01월전력수요예측전망
📅 작성일: 2018. 12.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2019년02월전력수요예측전망
📅 작성일: 2019. 1.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2019년03월전력수요예측전망
📅 작성일: 2019. 2.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2019년04월전력수요예측전망
📅 작성일: 2019. 3.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2019년05월전력수요예측전망 v1
📅 작성일: 2019. 4.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2019년06월전력수요예측전망 v1
📅 작성일: 2019. 5.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2019년07월전력수요예측전망 v2 (2)
📅 작성일: 2019. 6.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2019년08월전력수요예측전망 v4
📅 작성일: 2019. 7.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2019년09월전력수요예측전망 v1
📅 작성일: 2019. 8.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2019년10월전력수요예측전망 v1
📅 작성일: 2019. 9.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2019년11월전력수요예측전망 v1
📅 작성일: 2019. 10.
📊 섹션 수: 4
✅ 문서 구조화 완료!
📄 제목: 2019년12월전력수요예측전망 v1
📅 작성일: 2019. 11.
📊 섹션 수: 4


In [296]:
import json
def save_structured_doc(structured_doc, filename):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(structured_doc, f, ensure_ascii=False, indent=2)

# 
save_structured_doc(structured_doc_201901, "structured_doc_201901.json")
save_structured_doc(structured_doc_201902, "structured_doc_201902.json")
save_structured_doc(structured_doc_201903, "structured_doc_201903.json")
save_structured_doc(structured_doc_201904, "structured_doc_201904.json")
save_structured_doc(structured_doc_201905, "structured_doc_201905.json")
save_structured_doc(structured_doc_201906, "structured_doc_201906.json")
save_structured_doc(structured_doc_201907, "structured_doc_201907.json")
save_structured_doc(structured_doc_201908, "structured_doc_201908.json")
save_structured_doc(structured_doc_201909, "structured_doc_201909.json")
save_structured_doc(structured_doc_201910, "structured_doc_201910.json")
save_structured_doc(structured_doc_201911, "structured_doc_201911.json")
save_structured_doc(structured_doc_201912, "structured_doc_201912.json")